In [1]:
import numpy as np
import pandas as pd
from cmath import phase

In [2]:
'''
Una función que a partir del nombre del fichero npy carge los datos, los redimensione y los devuelva. 
Al estilo de la función get_data(). Devuelve "datos_bruto". 

Los test de esta función:   comprobar que devuelve un array 2D, 
                            comprobar que la segunda dimensión tiene tamaño 291, 
                            comprobar que la matriz tiene una parte real 
                            y que tiene una parte imaginaria.
'''

def get_data(url):
    try:
        diccionario = np.load(url, allow_pickle=True).item()
        data = diccionario.get('sweep_data').get('data')
        data = data.reshape(data.shape[0],data.shape[2])
    except:
        data = np.load(url)
    return data

In [3]:
assert(len(get_data('Materiales\C01_V01.npy').shape) == 2) #comprobar que devuelve un array 2D
assert(get_data('Materiales\C01_V01.npy').shape[1] == 291) #comprobar que la segunda dimensión tiene tamaño 291
assert((get_data('Materiales\C01_V01.npy').real.mean) != None) #comprobar que la matriz tiene una parte real
assert((get_data('Materiales\C01_V01.npy').imag.mean) != None) #comprobar que la matriz tiene una parte imaginaria
print(get_data('Materiales\C01_V01.npy').shape)

(300, 291)


In [4]:
'''
Una función get_modulo_fase() que a partir del array 2D anterior, obtenga un array 2D, con el doble de anchura. 
Por ejemplo, si "datos_bruto" es de 300x291, "modulo_fase" será de 600x291. 
Esa función obtendrá el módulo del número complejo y la fase del número complejo. 
    Módulo será una matriz de 300x291 y fase también. 
Será necesario concatenarlas "horizontalmente". 
Los test comprobarán las dimensiones y tamaños.
'''

def get_modulo_fase(data):
    modulo = abs(data)
    fase = []
    for i in data:
        fila = []
        for j in i:
            fila.append(phase(j)) #Fase
        fase.append(fila)
    fase = np.asarray(fase)
    modulo_fase = np.concatenate((modulo, fase), axis=0)
    return np.asarray(modulo_fase)
    #return modulo,fase
    

In [5]:
help(phase)

Help on built-in function phase in module cmath:

phase(z, /)
    Return argument, also known as the phase angle, of a complex.



In [6]:
data = get_data('Materiales\C01_V01.npy')
assert(get_modulo_fase(data).shape[0] >= 600) #Dimensión de tamaño >= 600
assert(get_modulo_fase(data).shape[1] == 291) #Dimensión de tamaño = 291


#modulo,fase = get_modulo_fase(data)

In [7]:
min(get_modulo_fase(data)[0])

35.6931365951495

In [8]:
max(get_modulo_fase(data)[0])

1465.3961921610141

In [9]:
'''
Una función que a partir de los datos "modulo_fase" devuelva la media. 
La comprobación sería que devuelva un array de 1 dimensión de tamaño 582 (291 x 2).
'''
def get_media(modulo_fase):
    traspuesta = np.transpose(modulo_fase) #Obtenemos la matriz traspuesta(291x600) de modulo_fase(600x291)
    
    #Separamos la traspuesta en modulo y fase
    t_modulo = traspuesta[:,:int(traspuesta.shape[1]/2)]
    t_fase = traspuesta[:,int(traspuesta.shape[1]/2):]
    
    media = []
    for i in t_modulo:
        media.append(i.mean())
    for j in t_fase:
        media.append(j.mean())
        
    return np.asarray(media) #Devuelve medias modulo y fase

In [10]:
modulo_fase = get_modulo_fase(data)
assert(get_media(modulo_fase).shape[0] == 582) #Dimensión de tamaño = 582
assert(len(get_media(modulo_fase).shape) == 1) #Array de 1 dimensión

In [11]:
'''
Función encargada de obtener las siguientes particiones separadas en entrenamiento (part_train) y test (part_test)

Partición 1
Entrenamiento: Carton Vista2-Vista10 + Plastico Vista2-Vista10 + Cristal Vista2-Vista10
Test: Carton Vista1 + Plastico Vista1 + Cristal Vista1

Partición 2
Entrenamiento: Carton Vista1,Vista3-Vista10 + Plastico Vista1,Vista3-Vista10 + Cristal Vista1,Vista3-Vista10
Test: Carton Vista2 + Plastico Vista2 + Cristal Vista2

...

Partición 10

TEST:
Sumando el número de filas de test1 y train1 de lo mismo que sumando el número de filas de test2 y train2.
Comprobar el tamaño de varias vistas igual a 582

'''

def particiones(materiales, objetos, vistas):
    
    part_train = [] #todos los entrenamientos
    part_test = [] #todos los test
    #part_train[0][0] se corresponde con part_test[0][0]

    z = 1
    for x in objetos: #Bucle para crear las 10 particiones
        print("Particion:", z)
        train = [] #conjunto entrenamiento
        test = [] #conjunto de pruebas
        for y in materiales: #Bucle de los materiales = 3
            for i in objetos: #Bucle de los objetos = 10
                #obj = [] #Lista de las vistas de un objeto
                for j in vistas: #Bucle de las vistas = 10
 
                    url = ('Materiales\%s%s_V%s.npy' % (y,i,j))
                    data = get_data(url)
                    modulo_fase = get_modulo_fase(data)
                    media = get_media(modulo_fase)
                    
                    #obj.append(media)
                    
                    if x != j: #TRAIN 
                        train.append(media)
                    else: #TEST
                        test.append(media)
                        
                        
        part_train.append(train) 
        part_test.append(test)
        
        z+=1


    part_train = np.asarray(part_train)
    part_test = np.asarray(part_test)

    return part_train, part_test

In [12]:
materiales = ['C','P','G'] #['C','P','G']
objetos = ['01','02','03','04','05','06','07','08','09','10'] #['01','02','03','04','05','06','07','08','09','10']
vistas = ['01','02','03','04','05','06','07','08','09','10'] #['01','02','03','04','05','06','07','08','09','10']

train, test = particiones(materiales, objetos, vistas)

Particion: 1
Particion: 2
Particion: 3
Particion: 4
Particion: 5
Particion: 6
Particion: 7
Particion: 8
Particion: 9
Particion: 10


In [13]:
len(train[0]), len(test[0]), len(train[1]), len(test[1]), len(train[9]), len(test[9]), len(train[0][0])

(270, 30, 270, 30, 270, 30, 582)

In [14]:
train[0].shape

(270, 582)

In [15]:
test[0].shape

(30, 582)

In [16]:
assert(len(train[0]) + len(test[0])) == (len(train[1]) + len(test[1])) #Train1 60 + Test1 30 = Train2 60 + Test2 30
assert(len(train[0]) + len(test[0])) == (len(train[9]) + len(test[9])) #Train1 60 + Test1 30 = Train10 60 + Test10 30
assert(len(train[0][0]) == 582) #582 son las caracteristicas de una vista
assert(len(train[0][0]) == len(train[0][9]) == len(test[5][7])) #Las vistas tinenen todas un tamaño de 582 (291 x 2)(modulo x fase)

In [17]:
np.save('train.npy', train)
np.save('test.npy', test)

In [18]:
def conjunto_datos(materiales, objetos, vistas):
    
    part_datos = [] #todos los datos

    datos = [] #conjunto datos
    for y in materiales: #Bucle de los materiales = 3
        for i in objetos: #Bucle de los objetos = 10
            #obj = [] #Lista de las vistas de un objeto
            for j in vistas: #Bucle de las vistas = 10

                url = ('Materiales\%s%s_V%s.npy' % (y,i,j))
                data = get_data(url)
                modulo_fase = get_modulo_fase(data)
                media = get_media(modulo_fase)

                datos.append(media)

    part_datos = np.asarray(datos) 


    return part_datos

In [19]:
datos = conjunto_datos(materiales, objetos, vistas)

In [20]:
np.save('conjunto_datos.npy', datos)